In [1]:
import numpy as np
import pandas as pd
# from dataset import getKlines

In [2]:
symbol_name = 'ETHBTC'

orders = pd.read_csv('data/Orders.csv')

In [3]:
def getKlinesOrdered(symbol_name):
    order = orders[orders['symbol'] == symbol_name]
    order.set_index('time_order', inplace = True)

    # data = getKlines(symbol_name)
    data = pd.read_feather('data/' + symbol_name + '.feather')
    data = data.set_index('time').sort_index()

    df = data.join(order, how = 'outer')
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace = True)
    df.ffill(inplace = True)
    df.dropna(inplace = True)
    # df.reset_index(drop = True, inplace = True)
    df = df.astype({'open': 'float', 'high': 'float', 'low': 'float',
                    'close': 'float', 'volume': 'float', 'act_price': 'float',
                    'limit_price': 'float', 'delta': 'float'})
    df['actived'] = np.where(df['type'] == 'Buy / Long',
                             np.where(df['act_price'] > df['low'], df['act_price'], np.nan),
                             np.where(df['act_price'] < df['high'], df['act_price'], np.nan))
    df['actived'].ffill(inplace = True)
    df['limited'] = np.where(df['type'] == 'Buy / Long',
                            np.where((df['limit_price'] > df['low']) & df['actived'].notna(),
                                     df['limit_price'], np.nan),
                            np.where((df['limit_price'] < df['high']) & df['actived'].notna(),
                                     df['limit_price'], np.nan))
    df['limited'].ffill(inplace = True)
    limited = df.dropna(subset = ['limited'])

    df = pd.concat([df[df['limited'].isna()], limited.head(1)])

    return df

df = getKlinesOrdered(symbol_name)

In [5]:
df_1 = df.copy()
df_1.dropna(subset = ['actived'], inplace = True)

In [53]:
# def getStopLoss(df):

# i = df_1.index[5]
for i in df_1.index:
    a = df_1.loc[:i, ['high', 'low', 'close', 'type', 'delta']]
    b = a.at[i, 'low']
    df_1.at[i, 'stoploss'] = b if b <= min(a['low'].to_list()) else np.nan

df_1#.head(5)

,open,high,low,close,volume,symbol,type,act_price,limit_price,delta,actived,limited,stoploss
2023-05-18 02:10:00,0.067117,0.067179,0.066982,0.066983,323.4893,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-18 02:15:00,0.066982,0.066995,0.066852,0.066852,442.4273,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-18 02:20:00,0.066851,0.066913,0.066850,0.066913,60.8293,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-18 02:25:00,0.066915,0.066950,0.066914,0.066937,37.4797,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-18 02:30:00,0.066939,0.066939,0.066848,0.066849,139.3865,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-22 17:00:00,0.067660,0.067733,0.067660,0.067710,267.7751,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-22 17:05:00,0.067720,0.067730,0.067650,0.067680,527.8267,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-22 17:10:00,0.067680,0.067700,0.067670,0.067700,153.9893,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
2023-05-22 17:15:00,0.067700,0.067700,0.067680,0.067680,57.7220,ETHBTC,Buy / Long,0.067082,0.066358,0.5,0.067082,NaN,NaN
